In [ ]:
import time

start_time_main = time.time()

import pandas as pd

import numpy as np

import datetime

from flask import Flask, jsonify, url_for, redirect, request

from flask_pymongo import PyMongo

from flask_restful import Api, Resource

In [ ]:
#********************************************** Obtain User Id Starts ************************************************

def ObtainUserId():  # in numpy.datetime64 format
    
    while True:
        try:
            UserId = input("Please enter UserId: ")
            UserId = int(UserId)
            break
        except ValueError:
            print("No valid integer! Please try again ...")
    print("Great, you successfully entered an integer!")
    
    return UserId

#********************************************** Obtain User Id Ends ************************************************


#********************************************** Obtain Date Starts ************************************************

def ObtainDateDt64():  # in numpy.datetime64 format
    isValid=False
    while not isValid:
        userIn = input("Type Date dd/mm/yy: ")
        try: # strptime throws an exception if the input doesn't match the pattern
            d = datetime.datetime.strptime(userIn, "%d/%m/%y")
            dt64=np.datetime64(d)
            isValid=True
        except:
            print ("Please try again!\n")
    return dt64

#********************************************** Obtain Date Ends ************************************************

In [ ]:
#********************************************** Import CSV Files Starts************************************************ //bad method; find more suitable

#def import_csv():

#************ Portfoliosdata Starts************

portfoliosdata= pd.read_csv('C:/Users/akans/OneDrive/Desktop/personal/project/PortfolioManagement-20190411T180751Z-001/PortfolioManagement/data/PortfoliosData.csv')
portfoliosdata['Date']= pd.to_datetime(portfoliosdata['Date'])

#************ Portfoliosdata Ends************


#************ SharesSold Starts************

sharessold= pd.read_csv('C:/Users/akans/OneDrive/Desktop/personal/project/PortfolioManagement-20190411T180751Z-001/PortfolioManagement/data/SharesSold2.csv')
sharessold['PurchaseDate']= pd.to_datetime(sharessold['PurchaseDate'], dayfirst=True)
sharessold['SellDate']= pd.to_datetime(sharessold['SellDate'], dayfirst=True)
sharessold.sort_values("SellDate", inplace=True)

#************ ShareSold Ends************


#************ SharesBought Starts************

sharesbought= pd.read_csv('C:/Users/akans/OneDrive/Desktop/personal/project/PortfolioManagement-20190411T180751Z-001/PortfolioManagement/data/SharesBought.csv')
sharesbought['PurchaseDate']= pd.to_datetime(sharesbought['PurchaseDate'], dayfirst=True)
sharesbought.sort_values("PurchaseDate", inplace=True)

#************ SharesBought Ends************


#************ SharesPrices Starts************

sharesprices= pd.read_csv('C:/Users/akans/OneDrive/Desktop/personal/project/PortfolioManagement-20190411T180751Z-001/PortfolioManagement/data/SharesPrices.csv')
sharesprices['Date']= pd.to_datetime(sharesprices['Date'], dayfirst=True)
sharesprices.sort_values("Date", inplace=True)

#************ SharesPrices Ends************

#return portfoliosdata, sharessold, sharesbought, sharesprices

#********************************************** Import CSV Files Ends************************************************

In [ ]:
#********************************************** Current Cash Entry Starts ************************************************

def GetCashForUser(df_cash, userid, givenDate):
    
    cash = df_cash.loc[(df_cash["UserId"] == userid) & (df_cash["Date"] == givenDate), "Cash"].values
    
    try:
        currentCash= cash[0]
        
    except IndexError:
        currentCash= 0;
    
    
    return currentCash

#********************************************** Current Cash Entry Ends ************************************************

In [ ]:
#********************************************** profitLossOnStocksSold Starts ************************************************

def GetProfitLossAfterStocksSoldOnGivenDate(df_profitLossOnStocksSold, userid, givenDate):
    
    price = df_profitLossOnStocksSold.loc[(df_profitLossOnStocksSold["UserId"] == userid) & (df_profitLossOnStocksSold["SellDate"] < givenDate), "Profit"].groupby(df_profitLossOnStocksSold["UserId"]).sum()
    
    return price

#********************************************** profitLossOnStocksSold Ends ************************************************

In [ ]:
#********************************************** GetPastHoldingsPurchaseForAUserOnGivenDate Starts ************************************************

def GetPastHoldingsPurchaseForAUserOnGivenDate(df_GetPastHoldingsPurchaseForAUserOnGivenDate, userid, givenDate):
    
    price = df_GetPastHoldingsPurchaseForAUserOnGivenDate.loc[(df_GetPastHoldingsPurchaseForAUserOnGivenDate["UserId"] == userid) & (df_GetPastHoldingsPurchaseForAUserOnGivenDate["PurchaseDate"] <= givenDate) & (df_GetPastHoldingsPurchaseForAUserOnGivenDate["SellDate"] >= givenDate), ['PurchasePrice', 'SellQty']]
    
    multiply= (price['PurchasePrice']* price['SellQty']).sum()
    return multiply

#********************************************** GetPastHoldingsPurchaseForAUserOnGivenDate Ends ************************************************

In [ ]:
#********************************************** GetPastHoldingsPurchaseForAUserOnGivenDate Starts ************************************************

def GetBuyingPriceForAUserOnGivenDate(df_GetBuyingPriceForAUserOnGivenDate, userid, givenDate):
    
    price = df_GetBuyingPriceForAUserOnGivenDate.loc[(df_GetBuyingPriceForAUserOnGivenDate["UserId"] == userid) & (df_GetBuyingPriceForAUserOnGivenDate["PurchaseDate"] <= givenDate), ['PurchasePrice', 'PurchaseQty']]
    
    multiply= (price['PurchasePrice']* price['PurchaseQty']).sum()
    return multiply

#********************************************** GetPastHoldingsPurchaseForAUserOnGivenDate Ends ************************************************

In [ ]:
#********************************************** GetCurrentInvestmentsActualPriceForAUserOnGivenDate Starts ************************************************

def GetCurrentInvestmentsActualPriceForAUserOnGivenDate(df_GetCurrentInvestmentsActualPriceForAUserOnGivenDate_SharesPrices, df_GetCurrentInvestmentsActualPriceForAUserOnGivenDate_SharesBought, userid, givenDate):
    
    a= df_GetCurrentInvestmentsActualPriceForAUserOnGivenDate_SharesPrices;
    b= df_GetCurrentInvestmentsActualPriceForAUserOnGivenDate_SharesBought;
    
    
    closing_price = a.loc[(a["Date"] == givenDate)]
    purchase_qty = b.loc[(b["UserId"] == userid) & (b["PurchaseDate"] <= givenDate)]
    
    merged= pd.merge(closing_price, purchase_qty, on='Symbol', how='inner') 
    
    multiply= (merged['ClosingPrice']* merged['PurchaseQty']).sum()
    
    return multiply

#********************************************** GetCurrentInvestmentsActualPriceForAUserOnGivenDate Ends ************************************************

In [ ]:
#********************************************** GetPastHoldingsActualWorthForAUserOnGivenDate Starts ************************************************

def GetPastHoldingsActualWorthForAUserOnGivenDate(df_GetPastHoldingsActualWorthForAUserOnGivenDate_SharesPrices, df_GetPastHoldingsActualWorthForAUserOnGivenDate_SharesSold, userid, givenDate):
    
    a= df_GetPastHoldingsActualWorthForAUserOnGivenDate_SharesPrices;
    b= df_GetPastHoldingsActualWorthForAUserOnGivenDate_SharesSold;
    
    
    closing_price = a.loc[(a["Date"] == givenDate)]
    sell_qty = b.loc[(b["UserId"] == userid) & (b["PurchaseDate"] <= givenDate) & (b["SellDate"] >= givenDate)]
    
    merged= pd.merge(closing_price, sell_qty, on='Symbol', how='inner')
    
    multiply= (merged['ClosingPrice']* merged['SellQty']).sum()
    return multiply

#********************************************** GetPastHoldingsActualWorthForAUserOnGivenDate Ends ************************************************

In [ ]:
def main():
    
    userid= ObtainUserId() 
    givenDate= ObtainDateDt64()

    start_time_mini = time.time()
    
    cash = GetCashForUser(portfoliosdata, userid, givenDate )
    print(cash)
    profitLossOnStocksSold = GetProfitLossAfterStocksSoldOnGivenDate(sharessold, userid, givenDate )
    networthPurchasePastHoldings = GetPastHoldingsPurchaseForAUserOnGivenDate(sharessold, userid, givenDate )  
    buyingPriceOnGivenDateForAUser = GetBuyingPriceForAUserOnGivenDate(sharesbought, userid, givenDate )  
    
    netCashOnADate = cash + profitLossOnStocksSold - networthPurchasePastHoldings - buyingPriceOnGivenDateForAUser;
    print (netCashOnADate)
    
    
    currentActualHoldingNetWorth = GetCurrentInvestmentsActualPriceForAUserOnGivenDate(sharesprices, sharesbought, userid, givenDate )  
    netWorthActualPastHoldings = GetPastHoldingsActualWorthForAUserOnGivenDate(sharesprices, sharessold, userid, givenDate )  
    
    currentHoldingWorth= currentActualHoldingNetWorth + netWorthActualPastHoldings;
    print (currentHoldingWorth)

    netWorth = netCashOnADate + currentHoldingWorth;
    print (netWorth)
    
    print('%s seconds for main' % (time.time() - start_time_main))
    print('%s seconds for mini' % (time.time() - start_time_mini))
 
if __name__ == "__main__":
    main()